Documentation at https://deepnote.com/viewer/github/project-chip/connectedhomeip/blob/master/docs/guides/repl/Matter_Basic_Interactions.ipynb
Unfortunately this documentation disappeared and I have no copy!

In [ ]:
# Clear any leftovers from the previous session
REALLY_CLEAR=False

if REALLY_CLEAR:
    import os, subprocess

    if os.path.isfile('repl.json'):
        os.remove('repl.json')

    # So that the all-clusters-app won't boot with stale prior state.
    # os.system('sudo rm -rf /tmp/chip_*')

In [ ]:
# Load the cluster and settings of the controler
import chip.native
import pkgutil
module = pkgutil.get_loader('chip.ChipReplStartup')
%run {module.path} --storagepath repl.json

import chip.clusters as Clusters

In [ ]:
import chip.clusters as Clusters

bridgeId = 3
lightId = 4

wifi_ssid = __REPLACE_ME__
wifi_credentials = __REPLACE_ME__

thread_dataset = b'__REPLACE_ME__'

In [ ]:
# retrieve the thread operational dataset by 'sudo ot-ctl dataset active -x'
if REALLY_CLEAR:
    # Use this if the bridge is not commissioned at all
    # devCtrl.CommissionThread(3840, 20202021, bridgeId, threadOperationalDataset=thread_dataset)
    # Use this when the bridge is already commissioned to a Thread network
    devCtrl.CommissionWithCode(setupPayload='MT:Y.K90AFN002J.F35E10',nodeid=bridgeId)
    # Use this when using the nRF7002dk
    devCtrl.CommissionWifi(3840, 20202021, bridgeId, ssid=wifi_ssid, credentials=wifi_credentials)


In [ ]:
if REALLY_CLEAR:
    # devCtrl.CommissionThread(3840, 20202021, lightId, threadOperationalDataset=thread_dataset)    
    devCtrl.CommissionWithCode(setupPayload='MT:6FCJ1AFN00KA0K7J020',nodeid=lightId)


In [ ]:
import asyncio
queue = asyncio.Queue()

import nest_asyncio
nest_asyncio.apply()

def mapPostureScoreLevelToHue(level):
    # Posture score range from 0 (good) to 255 (bad); 256 steps; stepping in positive direction
    # Hue range from 166 (good) to 216 (bad); 205 steps; stepping in negative direction; roll over at 0 to 255
    rangeMultiplicator = 205 / 256
    hueGoodPoint = 166
    hue = hueGoodPoint - (level * rangeMultiplicator)
    if hue < 0:
        hue = 255 + hue
    return hue
                           
                             

async def changeColors():
    print("changeColors")
    while True:
        levelValue = await queue.get()
        hueValue = mapPostureScoreLevelToHue(levelValue)
        print(f"get {levelValue}, set {hueValue}")
        await devCtrl.SendCommand(4, 1, Clusters.ColorControl.Commands.MoveToHue(hueValue, 0, 1, 0, 0))

async def subscribe():
    print("subscribe")
    reportingTimingParams = (0, 2) # MinInterval = 0s, MaxInterval = 2s
    subscription = await devCtrl.ReadAttribute(bridgeId, [3, Clusters.LevelControl.Attributes.CurrentLevel], reportInterval=reportingTimingParams, returnClusterObject=True)
    print("subscribed")

    def OnAttributeChangeCb(path: 'TypedAttributePath', transaction: 'SubscriptionTransaction'):  
        # levelValue = transaction.GetAttribute(path)
        # Does not work due to missing attributes. Got a Decoding Error object with incomplete values. 
        # Therefore get the first value of the error object (TLVValue) since we know this is the current level.
        # Should be fixed in bridge version 2
        levelValue = transaction.GetAttributes().get(path.Path.EndpointId).get(path.ClusterType).TLVValue.get(0)
        print(f"put {levelValue}")
        queue.put_nowait(levelValue)

    subscription.SetAttributeUpdateCallback(callback=OnAttributeChangeCb)
    
    while True:
        await asyncio.sleep(0)

loop = asyncio.get_event_loop()
loop.create_task(changeColors())
loop.create_task(subscribe())

# Note that to stop subscriptions, a restart of the Jupyter kernel is required.
#subscription.Shutdown()

In [ ]:
subscription.Shutdown()

Below are some commands for trying things out.

In [ ]:
await devCtrl.ReadAttribute(bridgeId, [('*')])

In [ ]:
await devCtrl.ReadAttribute(lightId, [('*')])

In [ ]:
# Search for commissionable devices
devices = devCtrl.DiscoverCommissionableNodes(filterType=chip.discovery.FilterType.LONG_DISCRIMINATOR, filter=discriminator, stopOnFirst=True, timeoutSecond=2)
devices

In [ ]:
# matterhelp(Clusters.ColorControl.Commands.MoveToHue)
# ./chip-tool colorcontrol move-to-hue Hue Direction TransitionTime OptionsMask OptionsOverride destination-id endpoint-idendpoint-id
# ./chip-tool colorcontrol move-to-hue 200 0         1              0           0               3              1
await devCtrl.SendCommand(lightId, 1, Clusters.ColorControl.Commands.MoveToHue(100, 0, 1, 0, 0))

In [ ]:
# ./chip-tool levelcontrol move-to-level Level TransitionTime OptionsMask OptionsOverride destination-id endpoint-id
# ./chip-tool levelcontrol move-to-level 254   0              0           0               3              1
await devCtrl.SendCommand(lightId, 1, Clusters.LevelControl.Commands.MoveToLevel(2, 1, 0, 0))

In [ ]:
await devCtrl.SendCommand(lightId, 1, Clusters.OnOff.Commands.On())


In [ ]:
matterhelp(devCtrl)